# Create Consumption Pricing objects in ADLSTE_COA

In [1]:
%connect name=Transcend

Password: ············


Success: 'Transcend' connection established and activated for user 'sh186014'


<br/><br/><br/>
## COA Populated Tables - Consumption Pricing

In [8]:
drop table adlste_coa_stg.stg_dat_consumption_vantage_units

Success: 19 rows affected

In [10]:
-- GTT Table
CREATE GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_consumption_vantage_units 
( Site_ID  VARCHAR(8) CHARACTER SET UNICODE 
, LogDate  DATE       FORMAT 'YYYY-MM-DD'
, LogHour  INTEGER
, CPU_VU   FLOAT
, IO_VU    FLOAT
) Primary Index (Site_ID, LogDate)
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [13]:
-- GTT Table
CREATE TABLE adlste_coa.coat_dat_consumption_vantage_units 
( Site_ID  VARCHAR(8) CHARACTER SET UNICODE 
, LogDate  DATE       FORMAT 'YYYY-MM-DD'
, LogHour  INTEGER
, CPU_VU   FLOAT
, IO_VU    FLOAT
, Run_Id   INTEGER
) Primary Index (Site_ID, LogDate);

Success: 0 rows affected

In [16]:
replace view adlste_coa.coa_dat_consumption_vantage_units
as locking row for access
select * from adlste_coa.coat_dat_consumption_vantage_units;

Success: 0 rows affected

In [17]:
REPLACE PROCEDURE adlste_coa.sp_dat_consumption_vantage_units ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coa_dat_consumption_vantage_units';
    SET callingsp = 'adlste_coa.sp_dat_consumption_vantage_units';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(LogDate) as StartDate
    ,max(LogDate) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_consumption_vantage_units
    Group by 1
    ;
    
    /* end setup audit process */
    
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START',  '') ;

    DELETE FROM adlste_coa.coat_dat_consumption_vantage_units WHERE (Site_ID, LogDate ) IN
    (Select Site_ID, LogDate From adlste_coa_stg.stg_dat_consumption_vantage_units );

    Insert into adlste_coa.coat_dat_consumption_vantage_units 
    SELECT
      Site_ID
    , LogDate
    , LogHour
    , CPU_VU
    ,IO_VU
    , :runid
    FROM adlste_coa_stg.stg_dat_consumption_vantage_units;

    delete from adlste_coa_stg.stg_dat_consumption_vantage_units;
        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END', '') ;

END;

Success: 0 rows affected

# Test:

In [20]:
call adlste_coa.sp_dat_consumption_vantage_units('test')

Success: 0 rows affected

In [11]:
insert into adlste_coa_stg.stg_dat_consumption_vantage_units 
select
 '{siteid}' as Site_ID,
 cast(coalesce(vu1.logdate, vu4.logdate) as format 'Y4-MM-DD') as LogDate,
 coalesce(vu1.loghr, vu4.loghr) as LogHour,
 zeroifnull(vu1.vantageunithrs) + zeroifnull(vu4.vantageunitutilityhrs) (format 'ZZ9.999') as CPU_VU,
 zeroifnull(vu1.vantageunitTB) + zeroifnull(vu4.vantageunitutilityTB) (format 'ZZ9.999') as IO_VU
from
    (select logdate,
     extract(hour from firstresptime) as LogHr,
     zeroifnull(sum(ampcputime + parsercputime + discputime)) / 3600 (format 'ZZ9.999') as VantageUnitHrs,
     zeroifnull(sum(reqiokb)) / (1024*1024*1024) (format 'ZZ9.999') as VantageUnitTB
    from
    pdcrinfo.dbqlogtbl_hst
    where logdate between DATE-91 and DATE-1
      and errorcode not in (2631, 2646, 3610, 3702, 3710, 3711, 5405, 7453, 7487, 7583, 7596, 9124, 9990)
      and username not in ('vcmuser','PDCRAccess','LockLogShredder','PDCRTPCD','console','tdap_admin','TDPUSER','tdwm',
                           'PDCRAdmin','SystemFe','PDCRCanary1M','PDCRCanary3M','td_ffe_svc_acct','PDCRCanary4M','PDCRCanary0M',
                           'PDCR users','PDCRCanary2M','TDMaps','SysAdmin','Crashdumps','Sys_Calendar')
    group by 1, 2) vu1
full outer join
    (select LogDate, LogHr,
     sum(Phase0IO) as TotalIOP0,
     sum(Phase1IO) as TotalIOP1,
     sum(Phase2IO) as TotalIOP2,
     sum(Phase3IO) as TotalIOP3,
     sum(Phase4IO) as TotalIOP4,
     sum(Phase0CPU) as TotalCPUP0,
     sum(Phase1CPU) as TotalCPUP1,
     sum(Phase2CPU) as TotalCPUP2,
     sum(Phase3CPU) as TotalCPUP3,
     sum(Phase4CPU) as TotalCPUP4,
     zeroifnull(TotalIOP0 + TotalIOP1  + TotalIOP2 + TotalIOP3 + TotalIOP4) / (1024*1024*1024) (format 'ZZ9.999') as VantageUnitUtilityTB,
     zeroifnull(TotalCPUP0 + TotalCPUP1  + TotalCPUP2 + TotalCPUP3 + TotalCPUP4)/3600 (format 'ZZ9.999') as VantageUnitUtilityHrs
    from
    (select
      cast(jobendtime as date) as LogDate,
      extract(hour from jobendtime) as LogHr,
      0 as Phase0CPU,
      0 as Phase0IO,
      case when utilityname = 'FASTLOAD' or utilityname = 'TPTLOAD' or (utilityname = 'FASTEXP' and fastexportnospool = 'Y')
      then phase1totalcputime
      else 0 end as Phase1CPU,
      case when utilityname = 'FASTLOAD' or utilityname = 'TPTLOAD' or (utilityname = 'FASTEXP' and fastexportnospool = 'Y')
      then phase1iokb
      else 0 end as Phase1IO,
      case when utilityname = 'FASTEXP' or utilityname = 'BAR'
      then phase2totalcputime
      else 0 end as Phase2CPU,
      case when utilityname = 'FASTEXP' or utilityname = 'BAR'
      then phase2iokb
      else 0 end as Phase2IO,
      case when utilityname = 'BAR'
      then phase3totalcputime
      else 0 end as Phase3CPU,
      case when utilityname = 'BAR'
      then phase3iokb
      else 0 end as Phase3IO,
      0 as Phase4CPU,
      0 as Phase4IO
    from pdcrinfo.dbqlutilitytbl_hst
    where logdate between DATE-91 and DATE-1
  ) dbu
group by 1, 2) vu4
on
vu1.logdate = vu4.logdate and
vu1.loghr = vu4.loghr


Success: 2182 rows affected

In [22]:
select top 10 * from adlste_coa.coa_dat_consumption_vantage_units

Site_ID,LogDate,LogHour,CPU_VU,IO_VU,Run_Id
{siteid},2020-09-17,11,285.82198444444504,168.9397226497531,2534
{siteid},2020-09-29,20,314.59422888888923,160.91878675296903,2534
{siteid},2020-08-21,23,156.209018888889,94.56823365017772,2534
{siteid},2020-08-01,22,185.10314888888908,104.85672608017921,2534
{siteid},2020-08-20,17,377.68926111111125,212.69554916024208,2534
{siteid},2020-09-21,14,294.4965133333336,154.59358675405383,2534
{siteid},2020-08-14,14,356.9863433333335,184.39835879579186,2534
{siteid},2020-10-05,19,295.17226111111165,140.17980228364468,2534
{siteid},2020-10-17,18,547.6712044444448,163.16909041628242,2534
{siteid},2020-09-30,0,279.6428177777778,140.65394676849246,2534
